In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/otto-recommender-system/sample_submission.csv
/kaggle/input/otto-recommender-system/test.jsonl
/kaggle/input/otto-recommender-system/train.jsonl
/kaggle/input/otto-pipeline2-lb-0-576/top_20_clicks_v6_3.pqt
/kaggle/input/otto-pipeline2-lb-0-576/top_20_clicks_v6_0.pqt
/kaggle/input/otto-pipeline2-lb-0-576/top_20_clicks_v6_2.pqt
/kaggle/input/otto-pipeline2-lb-0-576/top_15_buy2buy_v6_0.pqt
/kaggle/input/otto-pipeline2-lb-0-576/__results__.html
/kaggle/input/otto-pipeline2-lb-0-576/top_20_clicks_v6_1.pqt
/kaggle/input/otto-pipeline2-lb-0-576/top_15_carts_orders_v6_0.pqt
/kaggle/input/otto-pipeline2-lb-0-576/submission.csv
/kaggle/input/otto-pipeline2-lb-0-576/__notebook__.ipynb
/kaggle/input/otto-pipeline2-lb-0-576/__output__.json
/kaggle/input/otto-pipeline2-lb-0-576/top_15_carts_orders_v6_1.pqt
/kaggle/input/otto-pipeline2-lb-0-576/top_15_carts_orders_v6_2.pqt
/kaggle/input/otto-pipeline2-lb-0-576/top_15_carts_orders_v6_3.pqt
/kaggle/input/otto-pipeline2-lb-0-576/custom.css

In [2]:
import  itertools
from collections import Counter

In [3]:
train = pd.read_parquet('/kaggle/input/otto-train-and-test-data-for-local-validation/train.parquet')
val = pd.read_parquet('/kaggle/input/otto-train-and-test-data-for-local-validation/test.parquet')

In [4]:
val

,session,aid,ts,type
0,11098528,11830,1661119200,0
1,11098529,1105029,1661119200,0
2,11098530,264500,1661119200,0
3,11098530,264500,1661119288,0
4,11098530,409236,1661119369,0
...,...,...,...,...
7683572,12899774,33035,1661723968,0
7683573,12899775,1743151,1661723970,0
7683574,12899776,548599,1661723972,0
7683575,12899777,384045,1661723976,0


In [5]:
train

,session,aid,ts,type
0,0,1517085,1659304800,0
1,0,1563459,1659304904,0
2,0,1309446,1659367439,0
3,0,16246,1659367719,0
4,0,1781822,1659367871,0
...,...,...,...,...
163955175,11098523,175715,1661119197,0
163955176,11098524,1088524,1661119198,0
163955177,11098525,182927,1661119199,0
163955178,11098526,510055,1661119199,0


In [6]:
%%time
VER=6
DISK_PIECES=4
def pqt_to_dict(df):
    return df.groupby('aid_x').aid_y.apply(list).to_dict()

top_20_clicks = pqt_to_dict(pd.read_parquet(f'/kaggle/input/otto-pipeline2-lb-0-576/top_20_clicks_v6_0.pqt'))
for k in range(1, DISK_PIECES):
    top_20_clicks.update( pqt_to_dict( pd.read_parquet(f'/kaggle/input/otto-pipeline2-lb-0-576/top_20_clicks_v6_{k}.pqt')))
top_20_buys = pqt_to_dict( pd.read_parquet(f'/kaggle/input/otto-pipeline2-lb-0-576/top_15_carts_orders_v6_0.pqt'))
for k in range(1, DISK_PIECES):
    top_20_buys.update( pqt_to_dict( pd.read_parquet(f'/kaggle/input/otto-pipeline2-lb-0-576/top_15_carts_orders_v6_{k}.pqt')))
top_20_buy2buy = pqt_to_dict(pd.read_parquet(f'/kaggle/input/otto-pipeline2-lb-0-576/top_15_buy2buy_v6_0.pqt'))

top_clicks = val.loc[val['type']== 0, 'aid'].value_counts().index.values[:150]
top_carts = val.loc[val['type']== 1, 'aid'].value_counts().index.values[:150]
top_orders = val.loc[val['type']== 2, 'aid'].value_counts().index.values[:150]

print('Here are size of our 3 co-visitation matrices:')
print( len(top_20_clicks), len(top_20_buy2buy), len(top_20_buys))


Here are size of our 3 co-visitation matrices:
1837166 1168768 1837166
CPU times: user 1min 46s, sys: 8.56 s, total: 1min 54s
Wall time: 1min 54s


In [7]:
top_clicks

array([ 485256, 1460571,  108125, 1551213,   33343,  613493,  876493,
        152547,  184976, 1236775, 1406660,  836852,  331708, 1531805,
         29735,  554660,  634452,  986164,  959208,  832192,  332654,
        620545, 1645990,  166037,  102345, 1116095,  811371,  322370,
       1733943, 1043508, 1019736,  756588,  801774,  231487,  861401,
       1502122, 1030009,  329725, 1624436, 1022566, 1006198,  496180,
       1685214, 1853288,  530377,  659399, 1604220, 1102089, 1586171,
       1497089,  754412, 1765072, 1109824, 1264313,  881286, 1294924,
        199409, 1581568,  385065,  195381,  558573, 1196256, 1782099,
        612920,  435253,  923948,  508883, 1126038, 1647563, 1508062,
       1182614,  544144,  247240,  982423,   57315, 1628069,  674590,
       1798916,  892871,  909449,  337471,  721034, 1734305,   95488,
       1722991, 1754984,  819288, 1462420,  836707, 1796103,   42628,
        642804,  122983,  321547, 1278671,  714524, 1603001,  409620,
        636101, 1702

In [8]:
type_weight_multipliers = {0: 1, 1: 5, 2: 4}

def suggest_clicks(df):
    # USER HISTORY AIDS AND TYPES
    aids=df.aid.tolist()
    types = df.type.tolist()
    unique_aids = list(dict.fromkeys(aids[::-1] ))
    # RERANK CANDIDATES USING WEIGHTS
    if len(unique_aids)>=150:
        weights=np.logspace(0.1,1,len(aids),base=2, endpoint=True)-1
        aids_temp = Counter() 
        # RERANK BASED ON REPEAT ITEMS AND TYPE OF ITEMS
        for aid,w,t in zip(aids,weights,types): 
            aids_temp[aid] += w * type_weight_multipliers[t]
        sorted_aids = [k for k,v in aids_temp.most_common(150)]
        return sorted_aids
    # USE "CLICKS" CO-VISITATION MATRIX
    aids2 = list(itertools.chain(*[top_20_clicks[aid] for aid in unique_aids if aid in top_20_clicks]))
    aids3 = list(itertools.chain(*[top_20_buys[aid] for aid in unique_aids if aid in top_20_buys]))
    # RERANK CANDIDATES
    top_aids2 = [aid2 for aid2, cnt in Counter(aids2+aids3).most_common(150) if aid2 not in unique_aids]    
    result = unique_aids + top_aids2[:150 - len(unique_aids)]
    # USE TOP20 TEST CLICKS
    tops = set(list(top_clicks)) - set(result)
    return result + list(tops)[:150-len(result)]


def suggest_carts(df):
    # User history aids and types
    aids = df.aid.tolist()
    types = df.type.tolist()
    
    # UNIQUE AIDS AND UNIQUE BUYS
    unique_aids = list(dict.fromkeys(aids[::-1] ))
    df = df.loc[(df['type'] == 0)|(df['type'] == 1)]
    unique_buys = list(dict.fromkeys(df.aid.tolist()[::-1]))
    
    # Rerank candidates using weights
    if len(unique_aids) >= 150:
        weights=np.logspace(0.5,1,len(aids),base=2, endpoint=True)-1
        aids_temp = Counter() 
        
        # Rerank based on repeat items and types of items
        for aid,w,t in zip(aids,weights,types): 
            aids_temp[aid] += w * type_weight_multipliers[t]
        
        # Rerank candidates using"top_20_carts" co-visitation matrix
        aids2 = list(itertools.chain(*[top_20_buys[aid] for aid in unique_buys if aid in top_20_buys]))
        for aid in aids2: aids_temp[aid] += 0.1
        sorted_aids = [k for k,v in aids_temp.most_common(150)]
        return sorted_aids
    
    # Use "cart order" and "clicks" co-visitation matrices
    aids1 = list(itertools.chain(*[top_20_clicks[aid] for aid in unique_aids if aid in top_20_clicks]))
    aids2 = list(itertools.chain(*[top_20_buys[aid] for aid in unique_aids if aid in top_20_buys]))
    aids3 = list(itertools.chain(*[top_20_buy2buy[aid] for aid in unique_aids if aid in top_20_buy2buy]))
    
    # RERANK CANDIDATES
    top_aids2 = [aid2 for aid2, cnt in Counter(aids1+aids2+aids3).most_common(150) if aid2 not in unique_aids] 
    result = unique_aids + top_aids2[:150 - len(unique_aids)]
    
    # USE TOP20 TEST ORDERS
    tops = set(list(top_carts)) - set(result)

    return result + list(tops)[:150-len(result)]


def suggest_buys(df):
    # USER HISTORY AIDS AND TYPES
    aids=df.aid.tolist()
    types = df.type.tolist()
    # UNIQUE AIDS AND UNIQUE BUYS
    unique_aids = list(dict.fromkeys(aids[::-1] ))
    df = df.loc[(df['type']==1)|(df['type']==2)]
    unique_buys = list(dict.fromkeys( df.aid.tolist()[::-1] ))
    # RERANK CANDIDATES USING WEIGHTS
    if len(unique_aids)>=150:
        weights=np.logspace(0.5,1,len(aids),base=2, endpoint=True)-1
        aids_temp = Counter() 
        # RERANK BASED ON REPEAT ITEMS AND TYPE OF ITEMS
        for aid,w,t in zip(aids,weights,types): 
            aids_temp[aid] += w * type_weight_multipliers[t]
        # RERANK CANDIDATES USING "BUY2BUY" CO-VISITATION MATRIX
        aids3 = list(itertools.chain(*[top_20_buy2buy[aid] for aid in unique_buys if aid in top_20_buy2buy]))
        for aid in aids3: aids_temp[aid] += 0.1
        sorted_aids = [k for k,v in aids_temp.most_common(150)]
        return sorted_aids
    # USE "CART ORDER" CO-VISITATION MATRIX
    aids2 = list(itertools.chain(*[top_20_buys[aid] for aid in unique_aids if aid in top_20_buys]))
    # USE "BUY2BUY" CO-VISITATION MATRIX
    aids3 = list(itertools.chain(*[top_20_buy2buy[aid] for aid in unique_buys if aid in top_20_buy2buy]))
    # RERANK CANDIDATES
    top_aids2 = [aid2 for aid2, cnt in Counter(aids2+aids3).most_common(150) if aid2 not in unique_aids] 
    result = unique_aids + top_aids2[:150 - len(unique_aids)]
    # USE TOP20 TEST ORDERS
    
    tops = set(list(top_orders)) - set(result)

    return result + list(tops)[:150-len(result)]

In [9]:
%%time

pred_df_clicks = val.sort_values(["session", "ts"]).groupby(["session"]).apply(
    lambda x: suggest_clicks(x)
)


CPU times: user 6min 38s, sys: 10.3 s, total: 6min 48s
Wall time: 6min 48s


In [10]:
pred_df_clicks = pd.DataFrame(pred_df_clicks)
pred_df_clicks.reset_index(inplace=True)
pred_df_clicks = pred_df_clicks.rename(columns={0:'aids'})
pred_df_clicks = pred_df_clicks.iloc[:100000, :]
pred_df_clicks = pred_df_clicks.explode('aids')
pred_df_clicks['aids'] = pred_df_clicks['aids'].astype('int32')
pred_df_clicks['session'] = pred_df_clicks['session'].astype('int32')

pred_df_clicks.to_parquet('pred_df_clicks.pqt')

import gc
del pred_df_clicks
_=gc.collect()

In [11]:
%%time
pred_df_carts = val.sort_values(["session", "ts"]).groupby(["session"]).apply(
    lambda x: suggest_carts(x)
)

CPU times: user 29min 47s, sys: 29.3 s, total: 30min 17s
Wall time: 30min 14s


In [12]:
pred_df_carts = pd.DataFrame(pred_df_carts)
pred_df_carts.reset_index(inplace=True)
pred_df_carts = pred_df_carts.rename(columns={0:'aids'})

pred_df_carts = pred_df_carts.iloc[:100000, :]
pred_df_carts = pred_df_carts.explode('aids')
pred_df_carts['aids'] = pred_df_carts['aids'].astype('int32')
pred_df_carts['session'] = pred_df_carts['session'].astype('int32')
pred_df_carts.to_parquet('pred_df_carts.pqt')

del pred_df_carts
_ = gc.collect()

In [13]:

%%time
pred_df_buys = val.sort_values(["session", "ts"]).groupby(["session"]).apply(
    lambda x: suggest_buys(x)
)

CPU times: user 26min 30s, sys: 38.1 s, total: 27min 8s
Wall time: 27min 7s


In [14]:
pred_df_buys = pd.DataFrame(pred_df_buys)
pred_df_buys.reset_index(inplace=True)
pred_df_buys = pred_df_buys.rename(columns={0:'aids'})

pred_df_buys = pred_df_buys.iloc[:100000, :]
pred_df_buys = pred_df_buys.explode('aids')
pred_df_buys['aids'] = pred_df_buys['aids'].astype('int32')
pred_df_buys['session'] = pred_df_buys['session'].astype('int32')
pred_df_buys.to_parquet('pred_df_buys.pqt')
